<a href="https://colab.research.google.com/github/andrewsundberg/DS301/blob/master/Exam1Part2SundbergAndrew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from google.colab import files
uploaded = files.upload()

%matplotlib inline

Saving german_credit_data.csv to german_credit_data.csv


In [0]:
import io
gc = pd.read_csv(io.BytesIO(uploaded['german_credit_data.csv']))
gc.head()

# a) (10 points) try handling the missing values the way that you preserve more of your observations.

In [0]:
gc1 = gc
gc1['Age'] = gc1['Age'].fillna((gc1['Age'].mean()))
gc1['Job'] = gc1['Job'].fillna((gc1['Job'].mean()))
gc1['Credit amount'] = gc1['Credit amount'].fillna((gc1['Credit amount'].mean()))
gc1['Duration'] = gc1['Duration'].fillna((gc1['Duration'].mean()))
gc1 = gc1.fillna(gc1.mode().iloc[0])
gc2 = gc1.loc[:, gc1.columns != 'Unnamed: 0']
gc2.head()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,little,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,little,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad


# b) (20 points) Apply one-hot encoding or to objects columns. You can choose to apply label-encoding to columns that have a numerical order

In [0]:
 from sklearn.preprocessing import OneHotEncoder

categorical = ['Sex', 'Housing', 'Saving accounts', 'Checking account', 'Purpose']
gc3 = pd.get_dummies(gc2,prefix_sep = "__", columns = categorical)

# c) Apply train_test_split

In [0]:
X = gc3.loc[:, gc3.columns != 'Risk']
y = gc3['Risk']
print("X.shape=", X.shape)
print("y.shape=", y.shape)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 100)

X.shape= (1000, 24)
y.shape= (1000,)


# d) (20 points) fit all these following classifiers and compare accuracy results in the way that easy to read:
- Decision Trees, Naive Bayes, K-Nearest Neighbours, Logistic Regression, SVM

# Decision Trees

In [0]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
clf = DecisionTreeClassifier(max_depth = 1)
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:",accuracy_score(y_test,y_pred))

Accuracy: 0.73


# K-Nearest Neighbors

In [0]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [0]:
y_pred1 = knn.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred1)

0.65

# Logistic Regression

In [0]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(solver = 'lbfgs')
logreg.fit(X_train,y_train)
logreg.predict(X_test)
y_pred = logreg.predict(X_test)
y_true = y_test

In [0]:
accuracy_score(y_true, y_pred)

0.71

# SVM

In [0]:
from sklearn.svm import SVC
svm = SVC()
svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)
accuracy_score(y_true, y_pred)

0.73

# e) (10 points) Apply StandardScaler() to your features, re-run step d) and compare the result

In [0]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


# Decision Trees

In [0]:
dt_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_dt", DecisionTreeClassifier()),
    ])

dt_clf.fit(X_train,y_train)
y_pred = dt_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.5766666666666667

# K-Nearest Neighbors

In [0]:
knn_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_dt", KNeighborsClassifier()),
    ])

knn_clf.fit(X_train,y_train)
y_pred = knn_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.6633333333333333

# Logistic Regression

In [0]:
logreg_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_dt", LogisticRegression()),
    ])

logreg_clf.fit(X_train,y_train)
y_pred = logreg_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.71

# SVM

In [0]:
svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_dt", SVC()),
    ])

svm_clf.fit(X_train,y_train)
y_pred = svm_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.71

On average, the models created in part d were better than the models created in part e. The model with the best accuracy score was the Decision Tree in part d.

# f) (20 points) Choose the feature from either e) or d). Apply more advance advanced algorithms:

- RandomForestClassifier with 1000 trees, Adaboost and XGBoost.  

# Random Forest

In [0]:
from sklearn.ensemble import RandomForestClassifier

rnd_reg = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_reg.fit(X_train, y_train)
y_pred_rf = rnd_reg.predict(X_test)
accuracy_score(y_pred_rf, y_test)

0.7

# Adaboost

In [0]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeRegressor

ada_reg = AdaBoostClassifier(DecisionTreeClassifier(max_depth=5), n_estimators=200, learning_rate=0.5)
ada_reg.fit(X_train, y_train)
y_pred_ada=ada_reg.predict(X_test)
accuracy_score(y_pred_ada,y_test)

0.65

# XG Boost

In [0]:
from xgboost import XGBClassifier

xgb_reg = XGBClassifier()
xgb_reg.fit(X_train, y_train)
y_pred_xgb = xgb_reg.predict(X_test)
accuracy_score(y_pred_xgb,y_test)

0.6966666666666667

# g) (20 points) Try to use all algorithms from steps d) and f) above to combine in the VotingClassifier(), It's ok to use voting='hard'.

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()
voting_clf = VotingClassifier(
 estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
 voting='hard')
voting_clf.fit(X_train, y_train)


VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='warn', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     class_weight=None,
                                                     criterion='gini',...
                                        

In [0]:
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.71
RandomForestClassifier 0.6566666666666666
SVC 0.73
VotingClassifier 0.72
